# Comparative Toxicogenomics Database (CTD): Gene-Chemical Interactions

Created by: Charles Dai <br>
Credit to: Moshe Silverstein

Data Source: http://ctdbase.org/downloads

In [1]:
# appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [2]:
import sys
import os
from datetime import date

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import harmonizome.utility_functions as uf
import harmonizome.lookup as lookup

In [3]:
# from clustergrammer_widget import *
# net = Network(clustergrammer_widget)

In [4]:
%load_ext autoreload
%autoreload 2

### Notebook Information

In [5]:
print('This notebook was run on:', date.today(), '\nPython version:', sys.version)

This notebook was run on: 2020-06-28 
Python version: 3.8.0 (default, Oct 28 2019, 16:14:01) 
[GCC 8.3.0]


# Initialization

### Load Mapping Dictionaries

In [6]:
symbol_lookup, geneid_lookup = lookup.get_lookups()

Gathering sources: 100%|██████████| 3/3 [00:09<00:00,  3.19s/it]


### Output Path

In [7]:
output_name = 'ctd_chemical'

path = 'Output/CTD-Chemical'
if not os.path.exists(path):
    os.makedirs(path)

In [8]:
%%appyter hide_code
{% do SectionField(
    name='data',
    title='Load Data',
    subtitle='Upload Files from the Comparative Toxicogenomics Database',
) %}

# Load Data

In [24]:
%%appyter code_exec

df = pd.read_csv({{FileField(
    constraint='.*\.tsv.gz$',
    name='chemical_genes', 
    label='Chemical-Gene Interactions (tsv.gz)', 
    default='Input/CTD/CTD_chem_gene_ixns.tsv.gz',
    section='data')
}}, sep='\t', skiprows=[x for x in range(27)] + [28], 
    usecols=['# ChemicalName', 'GeneSymbol', 'OrganismID', 'GeneForms'])

```python
df = pd.read_csv('Input/CTD/CTD_chem_gene_ixns.tsv.gz', sep='\t', skiprows=[x for x in range(27)] + [28],
    usecols=['# ChemicalName', 'GeneSymbol', 'OrganismID', 'GeneForms'])
```

In [25]:
df.head()

,# ChemicalName,GeneSymbol,GeneForms,OrganismID
0,10074-G5,AR,protein,9606.0
1,10074-G5,AR,protein,9606.0
2,10074-G5,AR,protein,9606.0
3,10074-G5,AR,protein,9606.0
4,10074-G5,EPHB2,protein,9606.0


In [26]:
df.shape

(1961543, 4)

# Pre-process Data

## Get Relevant Data

In [27]:
# Get only relevant species
df = df[np.logical_or.reduce([
    df['OrganismID'] == 9606, # human
    df['OrganismID'] == 10090, # mouse
    df['OrganismID'] == 10116 # rat
])]

In [28]:
# Get only interactions that are gene or protein type
df = df[np.logical_or(
    df['GeneForms'] == 'protein',
    df['GeneForms'] == 'gene'
)]
df.head()

,# ChemicalName,GeneSymbol,GeneForms,OrganismID
0,10074-G5,AR,protein,9606.0
1,10074-G5,AR,protein,9606.0
2,10074-G5,AR,protein,9606.0
3,10074-G5,AR,protein,9606.0
4,10074-G5,EPHB2,protein,9606.0


In [29]:
df = df[['GeneSymbol', '# ChemicalName']]
df.columns = ['Gene Symbol', 'Chemical Name']
df = df.set_index('Gene Symbol')
df.head()

,Chemical Name
Gene Symbol,
AR,10074-G5
AR,10074-G5
AR,10074-G5
AR,10074-G5
EPHB2,10074-G5


# Filter Data

## Map Gene Symbols to Up-to-date Approved Gene Symbols

In [30]:
df = uf.map_symbols(df, symbol_lookup, remove_duplicates=True)
df.shape

100%|██████████| 564734/564734 [00:00<00:00, 780891.63it/s]


(214780, 1)

# Analyze Data

## Create Binary Matrix

In [ ]:
binary_matrix = uf.binary_matrix(df)
binary_matrix.head()

In [ ]:
binary_matrix.shape

In [ ]:
uf.save_data(binary_matrix, path, output_name + '_binary_matrix', 
            compression='npz', dtype=np.uint8)

## Create Gene List

In [ ]:
gene_list = uf.gene_list(binary_matrix, geneid_lookup)
gene_list.head()

In [ ]:
gene_list.shape

In [ ]:
uf.save_data(gene_list, path, output_name + '_gene_list',
            ext='tsv', compression='gzip', index=False)

## Create Attribute List

In [ ]:
attribute_list = uf.attribute_list(binary_matrix)
attribute_list.head()

In [ ]:
attribute_list.shape

In [ ]:
uf.save_data(attribute_list, path, output_name + '_attribute_list',
            ext='tsv', compression='gzip')

## Create Gene and Attribute Set Libraries

In [ ]:
uf.save_setlib(binary_matrix, 'gene', 'up', path, output_name + '_gene_up_set')

In [ ]:
uf.save_setlib(binary_matrix, 'attribute', 'up', path, 
                           output_name + '_attribute_up_set')

## Create Attribute Similarity Matrix

In [ ]:
attribute_similarity_matrix = uf.similarity_matrix(binary_matrix.T, 'jaccard', sparse=True)
attribute_similarity_matrix.head()

In [ ]:
uf.save_data(attribute_similarity_matrix, path,
            output_name + '_attribute_similarity_matrix', 
            compression='npz', symmetric=True, dtype=np.float32)

In [ ]:
# net.load_df(attribute_similarity_matrix.iloc[:,:].copy())
# net.filter_N_top('row', rank_type='sum', N_top=300)
# net.cluster()
# net.widget()

## Create Gene Similarity Matrix

In [ ]:
gene_similarity_matrix = uf.similarity_matrix(binary_matrix, 'jaccard', sparse=True)
gene_similarity_matrix.head()

In [ ]:
uf.save_data(gene_similarity_matrix, path, 
            output_name + '_gene_similarity_matrix',
            compression='npz', symmetric=True, dtype=np.float32)

## Create Gene-Attribute Edge List

In [ ]:
edge_list = uf(binary_matrix)
uf.save_data(edge_list, path, output_name + 'edge_list', 
        ext='tsv', compression='gzip')

# Create Downloadable Save File

In [ ]:
uf.archive(path)

### Link to download output files: [click here](./output_archive.zip)